In [ ]:
!pip install torch transformers datasets
!pip install gliner  # Install GLiNER library.
!pip install gradio # Install gradio library
import gliner
print(gliner.__version__)  # Controlla se è installato correttamente
!pip install gradio

In [2]:
from typing import Dict, Union
from gliner import GLiNER
import gradio as gr # importing gradio library
import gradio as gr
import os

# Change "model/" to the correct model identifier or path.
# If you have downloaded the model locally, provide the full path to the model directory.
# If you intend to load a model from the Hugging Face model hub, use the model identifier (e.g., 'urchade/gliner_multi-v2.1')
#model = GLiNER.from_pretrained("./", load_tokenizer=True)
#model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1", load_tokenizer=True)
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
# ... (rest of your code) ...

#model = GLiNER.from_pretrained("model/", load_tokenizer=True)
# ... (rest of your code) ...


#model = GLiNER.from_pretrained("model/", load_tokenizer=True)

examples = [
    [
        "Count Leopold.",
        "person, book, location, date, actor, character",
        0.3,
        True,
    ],
    [
        """
Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.
-------
* Data Scientist, Data Analyst, or Data Engineer with 1+ years of experience.
* Experience with technologies such as Docker, Kubernetes, or Kubeflow
* Machine Learning experience preferred
* Experience with programming languages such as Python, C++, or SQL preferred
* Experience with technologies such as Databricks, Qlik, TensorFlow, PyTorch, Python, Dash, Pandas, or NumPy preferred
* BA or BS degree
* Active Secret OR Active Top Secret or Active TS/SCI clearance
""",
        "software package, programing language, software tool, degree, job title",
        0.3,
        False,
    ],
    [
        "However, both models lack other frequent DM symptoms including the fibre-type dependent atrophy, myotonia, cataract and male-infertility.",
        "disease, symptom",
        0.3,
        False,
    ],
    [
        "Synergy between signal transduction pathways is obligatory for expression of c-fos in B and T cell lines: implication for c-fos control via surface immunoglobulin and T cell antigen receptors.",
        "DNA, RNA, cell line, cell type, protein",
        0.3,
        False,
    ],
    [
        "The choice of the encoder and decoder modules of dnpg can be quite flexible, for instance long short term memory networks (lstm) or convolutional neural network (cnn).",
        "short acronym, long acronym",
        0.3,
        False,
    ],
    [
        "Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris.",
        "person, company, location, airplane",
        0.3,
        True,
    ],
    [
        "Feldman is a contributor to NBC Sports Boston's ``State of the Revs`` and ``Revolution Postgame Live`` programs as well as to 98.5 the SportsHub, SiriusXM FC's MLS coverage and to other New England and national radio outlets and podcasts.",
        "person, company, location",
        0.3,
        False,
    ],
    [
        "On 25 July 1948, on the 39th anniversary of Bleriot's crossing of the English Channel, the Type 618 Nene-Viking flew Heathrow to Paris (Villacoublay) in the morning carrying letters to Bleriot's widow and son (secretary of the FAI), who met it at the airport.",
        "date, location, person, organization",
        0.3,
        False,
    ],
    [
        "Leo & Ian won the 1962 Bathurst Six Hour Classic at Mount Panorama driving a Daimler SP250 sports car, (that year the 500 mile race for touring cars were held at Phillip Island)",
        "person, date, location, organization, competition",
        0.3,
        False,
    ],
    [
        "The Shore Line route of the CNS & M until 1955 served, from south to north, the Illinois communities of Chicago, Evanston, Wilmette, Kenilworth, Winnetka, Glencoe, Highland Park, Highwood, Fort Sheridan, Lake Forest, Lake Bluff, North Chicago, Waukegan, Zion, and Winthrop Harbor as well as Kenosha, Racine, and Milwaukee (the ``KRM'') in Wisconsin.",
        "location, organization, date",
        0.3,
        False,
    ],
    [
        "Comet C/2006 M4 (SWAN) is a non-periodic comet discovered in late June 2006 by Robert D. Matson of Irvine, California and Michael Mattiazzo of Adelaide, South Australia in publicly available images of the Solar and Heliospheric Observatory (SOHO).",
        "person, organization, date, location",
        0.3,
        False,
    ],
    [
        "From November 29, 2011 to March 31, 2012, Karimloo returned to ``Les Misérables`` to play the lead role of Jean Valjean at The Queen's Theatre, London, for which he won the 2013 Theatregoers' Choice Award for Best Takeover in a Role.",
        "person, actor, award, date, location",
        0.3,
        False,
    ],
    [
        "A Mexicali health clinic supported by former Baja California gubernatorial candidate Enrique Acosta Fregoso (PRI) was closed on June 15 after selling a supposed COVID-19 ``cure'' for between MXN $10,000 and $50,000.",
        "location, organization, person, date, currency",
        0.3,
        False,
    ],
    [
        "Built in 1793, it was the home of Mary Young Pickersgill when she moved to Baltimore in 1806 and the location where she later sewed the ``Star Spangled Banner'', in 1813, the huge out-sized garrison flag that flew over Fort McHenry at Whetstone Point in Baltimore Harbor in the summer of 1814 during the British Royal Navy attack in the Battle of Baltimore during the War of 1812.",
        "date, person, location, organization, event, flag",
        0.3,
        False,
    ],
]


def ner(
    text, labels: str, threshold: float, nested_ner: bool
) -> Dict[str, Union[str, int, float]]:
    labels = labels.split(",")
    return {
        "text": text,
        "entities": [
            {
                "entity": entity["label"],
                "word": entity["text"],
                "start": entity["start"],
                "end": entity["end"],
                "score": 0,
            }
            for entity in model.predict_entities(
                text, labels, flat_ner=not nested_ner, threshold=threshold
            )
        ],
    }


with gr.Blocks(title="GLiNER-M-v2.1") as demo:
    gr.Markdown(
        """
        # GLiNER-base
        GLiNER is a Named Entity Recognition (NER) model capable of identifying any entity type using a bidirectional transformer encoder (BERT-like). It provides a practical alternative to traditional NER models, which are limited to predefined entities, and Large Language Models (LLMs) that, despite their flexibility, are costly and large for resource-constrained scenarios.
        ## Links
        * Model: https://huggingface.co/urchade/gliner_multi-v2.1
        * All GLiNER models: https://huggingface.co/models?library=gliner
        * Paper: https://arxiv.org/abs/2311.08526
        * Repository: https://github.com/urchade/GLiNER
        """
    )
    with gr.Accordion("How to run this model locally", open=False):
        gr.Markdown(
            """
            ## Installation
            To use this model, you must install the GLiNER Python library:
            ```
            !pip install gliner
            ```

            ## Usage
            Once you've downloaded the GLiNER library, you can import the GLiNER class. You can then load this model using `GLiNER.from_pretrained` and predict entities with `predict_entities`.
            """
        )
        gr.Code(
            '''
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""
labels = ["person", "award", "date", "competitions", "teams"]
entities = model.predict_entities(text, labels)
for entity in entities:
    print(entity["text"], "=>", entity["label"])
            ''',
            language="python",
        )
        gr.Code(
            """
Cristiano Ronaldo dos Santos Aveiro => person
5 February 1985 => date
Al Nassr => teams
Portugal national team => teams
Ballon d'Or => award
UEFA Men's Player of the Year Awards => award
European Golden Shoes => award
UEFA Champions Leagues => competitions
UEFA European Championship => competitions
UEFA Nations League => competitions
Champions League => competitions
European Championship => competitions
            """
        )

    input_text = gr.Textbox(
        value=examples[0][0], label="Text input", placeholder="Enter your text here"
    )
    with gr.Row() as row:
        labels = gr.Textbox(
            value=examples[0][1],
            label="Labels",
            placeholder="Enter your labels here (comma separated)",
            scale=2,
        )
        threshold = gr.Slider(
            0,
            1,
            value=0.3,
            step=0.01,
            label="Threshold",
            info="Lower the threshold to increase how many entities get predicted.",
            scale=1,
        )
        nested_ner = gr.Checkbox(
            value=examples[0][2],
            label="Nested NER",
            info="Allow for nested NER?",
            scale=0,
        )
    output = gr.HighlightedText(label="Predicted Entities")
    submit_btn = gr.Button("Submit")
    examples = gr.Examples(
        examples,
        fn=ner,
        inputs=[input_text, labels, threshold, nested_ner],
        outputs=output,
        cache_examples=True,
    )

    # Submitting
    input_text.submit(
        fn=ner, inputs=[input_text, labels, threshold, nested_ner], outputs=output
    )
    labels.submit(
        fn=ner, inputs=[input_text, labels, threshold, nested_ner], outputs=output
    )
    threshold.release(
        fn=ner, inputs=[input_text, labels, threshold, nested_ner], outputs=output
    )
    submit_btn.click(
        fn=ner, inputs=[input_text, labels, threshold, nested_ner], outputs=output
    )
    nested_ner.change(
        fn=ner, inputs=[input_text, labels, threshold, nested_ner], outputs=output
    )

demo.queue()
demo.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Caching examples at: '/content/.gradio/cached_examples/14'
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7fe33b8d9908492a44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 1600 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7fe33b8d9908492a44.gradio.live


In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from typing import Dict, Union
from gliner import GLiNER
import gradio as gr
import os

# Carica il modello GLiNER
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

# Funzione per estrarre il testo da un file
def extract_text_from_file(file_obj):
    if file_obj is None:
        return ""
    file_path = file_obj.name
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except Exception as e:
        return f"Errore nella lettura del file: {str(e)}"

# Funzione NER che processa il testo
def ner(
    text, labels: str, threshold: float, nested_ner: bool
) -> Dict[str, Union[str, int, float]]:
    labels = labels.split(",")
    return {
        "text": text,
        "entities": [
            {
                "entity": entity["label"],
                "word": entity["text"],
                "start": entity["start"],
                "end": entity["end"],
                "score": 0,
            }
            for entity in model.predict_entities(
                text, labels, flat_ner=not nested_ner, threshold=threshold
            )
        ],
    }

# Funzione che gestisce il caricamento del file e chiama il NER
def process_file(file_obj, labels, threshold, nested_ner):
    if file_obj is None:
        return "Nessun file caricato", None

    text = extract_text_from_file(file_obj)
    if text.startswith("Errore"):
        return text, None

    result = ner(text, labels, threshold, nested_ner)
    return text, result

examples = [
    [
        "Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.",
        "person, book, location, date, actor, character",
        0.3,
        True,
    ],
    # Altri esempi...
]

with gr.Blocks(title="GLiNER-M-v2.1 con caricamento file") as demo:
    gr.Markdown(
        """
        # GLiNER con caricamento file
        Questa versione ti permette di caricare un file di testo (.txt) per analizzare le entità.
        """
    )

    with gr.Tab("Caricamento file"):
        file_input = gr.File(label="Carica un file .txt", file_types=[".txt"])
        with gr.Row() as row:
            labels = gr.Textbox(
                value="person, location, date, organization, stazione_appaltante, ente_pubblico, RUP, direzione_lavori, operatore_economico, fornitore, codice_CIG, codice_CUP, numero_gara, numero_lotto, numero_determina, codice_CPV, data_pubblicazione, data_scadenza, data_aggiudicazione, durata_contratto, importo_base_asta, importo_aggiudicazione, ribasso, cauzione, procedura_gara, criterio_aggiudicazione, categoria_lavori, categoria_servizi, normativa, decreto, articolo_legge, codice_contratti, indirizzo, comune, provincia, CAP, requisiti_partecipazione, documenti_richiesti, termini_pagamento, penali",
#               value="person, location, date, organization",
                label="Etichette",
                placeholder="Inserisci le etichette separate da virgola",
                scale=2,
            )
            threshold = gr.Slider(
                0,
                1,
                value=0.3,
                step=0.01,
                label="Soglia",
                info="Abbassa la soglia per aumentare il numero di entità rilevate.",
                scale=1,
            )
            nested_ner = gr.Checkbox(
                value=True,
                label="NER Annidate",
                info="Permetti entità annidate?",
                scale=0,
            )
        submit_btn = gr.Button("Analizza File")
        text_display = gr.Textbox(label="Testo Caricato", lines=10)
        output = gr.HighlightedText(label="Entità Rilevate")

    with gr.Tab("Input Manuale"):
        input_text = gr.Textbox(
            value=examples[0][0], label="Testo di input", placeholder="Inserisci il tuo testo qui", lines=10
        )
        with gr.Row() as row:
            manual_labels = gr.Textbox(
                value=examples[0][1],
                label="Etichette",
                placeholder="Inserisci le etichette separate da virgola",
                scale=2,
            )
            manual_threshold = gr.Slider(
                0,
                1,
                value=0.3,
                step=0.01,
                label="Soglia",
                info="Abbassa la soglia per aumentare il numero di entità rilevate.",
                scale=1,
            )
            manual_nested_ner = gr.Checkbox(
                value=examples[0][3],
                label="NER Annidate",
                info="Permetti entità annidate?",
                scale=0,
            )
        manual_submit_btn = gr.Button("Analizza Testo")
        manual_output = gr.HighlightedText(label="Entità Rilevate")

        gr.Examples(
            examples,
            fn=ner,
            inputs=[input_text, manual_labels, manual_threshold, manual_nested_ner],
            outputs=manual_output,
            cache_examples=True,
        )

    # Eventi per il caricamento del file
    submit_btn.click(
        fn=process_file,
        inputs=[file_input, labels, threshold, nested_ner],
        outputs=[text_display, output]
    )

    # Eventi per l'input manuale
    manual_submit_btn.click(
        fn=ner,
        inputs=[input_text, manual_labels, manual_threshold, manual_nested_ner],
        outputs=manual_output
    )

demo.queue()
demo.launch(debug=True)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Caching examples at: '/content/.gradio/cached_examples/54'
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a8b5f0bcf0d127c40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


----------------------------------------------

In [52]:
!pip install --upgrade gliner

In [55]:
from typing import Dict, Union, List
from gliner import GLiNER
import gradio as gr
import os
import re
from collections import defaultdict

# Carica il modello GLiNER
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
# Se hai bisogno di accedere al tokenizer dopo aver caricato il modello
#tokenizer = model.get_tokenizer() # Use get_tokenizer() to access the tokenizer
# Funzione per estrarre il testo da un file
def extract_text_from_file(file_obj):
    if file_obj is None:
        return ""
    file_path = file_obj.name
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except Exception as e:
        return f"Errore nella lettura del file: {str(e)}"

# 1️⃣ Suddivisione intelligente del testo basata sulla struttura dei bandi
def split_text_into_sections(text):
    sections = re.split(r'(?=Articolo\s+\d+|Sezione\s+\w+|Capitolo\s+\w+|Oggetto\s+del\s+Bando)', text)
    return [section.strip() for section in sections if section.strip()]

# 2️⃣ Sliding Window per gestire sezioni troppo lunghe
def sliding_window(text, max_len=512, overlap=100):
    words = text.split()
    segments = []
    for i in range(0, len(words), max_len - overlap):
        segments.append(" ".join(words[i:i + max_len]))
    return segments
"""
# 3️⃣ Post-processing: Unire entità frammentate o duplicate
def merge_entities(results):
    merged = []
    seen = set()

    if not isinstance(results, list):
        print("⚠️ ERRORE: merge_entities ha ricevuto un tipo sbagliato:", type(results))
        return []

    for res in results:
        if not isinstance(res, list):  # Verifica che sia una lista di dizionari
            print("⚠️ ERRORE: Elemento in results non è una lista:", type(res))
            continue

        for entity in res:
            if not isinstance(entity, dict):  # Verifica che sia un dizionario
                print("⚠️ ERRORE: Elemento in res non è un dizionario:", type(entity), "Valore:", entity)
                continue

            # Controllo chiavi presenti nel dizionario
            if not all(k in entity for k in ["label", "text", "start", "end"]):
                print("⚠️ ERRORE: Entità mancante di chiavi richieste:", entity)
                continue

            key = (entity["label"], entity["text"], entity["start"], entity["end"])
            if key not in seen:
                merged.append({
                    "entity": entity["label"],
                    "word": entity["text"],
                    "start": entity["start"],
                    "end": entity["end"],
                    "score": 0
                })
                seen.add(key)

    return merged

# 3️⃣ Post-processing: Unire entità frammentate o duplicate in un formato compatibile con Gradio
def merge_entities(results):
    merged = []
    seen = set()  # Per evitare duplicati

    # Se results è già una lista di dizionari (non una lista di liste)
    if results and isinstance(results[0], dict):
        for entity in results:
            key = (entity["entity"], entity["word"], entity["start"], entity["end"])
            if key not in seen:
                merged.append(entity)
                seen.add(key)
    else:
        # Caso originale - results è una lista di liste di dizionari
        for res in results:
            for entity in res:
                key = (entity["entity"], entity["word"], entity["start"], entity["end"])
                if key not in seen:
                    merged.append(entity)
                    seen.add(key)

    return merged  # Ritorna in formato lista di dizionari
"""
# 3️⃣ Post-processing: Unire entità frammentate o duplicate in un formato compatibile con Gradio
def merge_entities(results):
    merged = []
    seen = set()  # Per evitare duplicati

    # Verifico che tipo di dato è results
    if not results:
        return []

    # Se results contiene liste o dizionari
    for item in results:
        # Se item è un dizionario direttamente
        if isinstance(item, dict) and "entity" in item and "word" in item and "start" in item and "end" in item:
            key = (item["entity"], item["word"], item["start"], item["end"])
            if key not in seen:
                merged.append(item)
                seen.add(key)
        # Se item è una lista
        elif isinstance(item, list):
            for entity in item:
                if isinstance(entity, dict) and "entity" in entity and "word" in entity and "start" in entity and "end" in entity:
                    key = (entity["entity"], entity["word"], entity["start"], entity["end"])
                    if key not in seen:
                        merged.append(entity)
                        seen.add(key)

    return merged  # Ritorna in formato lista di dizionari
"""
# 4️⃣ Funzione NER aggiornata con approccio ibrido
def ner(text, labels: str, threshold: float, nested_ner: bool):
    labels = labels.split(",")
    sections = split_text_into_sections(text)  # Suddividi il testo in sezioni
    all_entities = []

    for section in sections:
        if len(section.split()) > 384:
            segments = sliding_window(section)
        else:
            segments = [section]

        for segment in segments:
            entities = model.predict_entities(segment, labels, flat_ner=not nested_ner, threshold=threshold)

            # DEBUG: Stampiamo il formato dei dati
            print("🔹 Risultato predict_entities:", entities)

            if not isinstance(entities, list):
                print("⚠️ ERRORE: Il modello ha restituito un tipo sbagliato:", type(entities))
                continue

            formatted_entities = [
                {
                    "entity": entity["label"],
                    "word": entity["text"],
                    "start": entity["start"],
                    "end": entity["end"],
                    "score": 0
                }
                for entity in entities if isinstance(entity, dict) and "label" in entity
            ]

            all_entities.append(formatted_entities)

    merged_entities = merge_entities(all_entities)
    return merged_entities
"""
"""
# 4️⃣ Funzione NER aggiornata con approccio ibrido
def ner(text, labels: str, threshold: float, nested_ner: bool):
    labels = labels.split(",")
    sections = split_text_into_sections(text)  # Suddividi il testo in sezioni
    all_results = []  # Modifica qui - raccogliamo i risultati di ciascun segmento

    for section in sections:
        if len(section.split()) > 384:  # Se la sezione è lunga, usa sliding window
            segments = sliding_window(section)
        else:
            segments = [section]

        section_entities = []  # Entità per questa sezione
        for segment in segments:
            entities = model.predict_entities(segment, labels, flat_ner=not nested_ner, threshold=threshold)
            formatted_entities = [
                {
                    "entity": entity["label"],
                    "word": entity["text"],
                    "start": entity["start"],
                    "end": entity["end"],
                    "score": 0
                }
                for entity in entities
            ]
            section_entities.append(formatted_entities)  # Aggiungi come lista separata

        all_results.append(section_entities)  # Aggiungi i risultati di questa sezione

    # Unisci entità duplicate o frammentate
    merged_entities = merge_entities(all_results)

    return merged_entities  # Adesso il formato è compatibile con Gradio HighlightedText
"""

# 4️⃣ Funzione NER aggiornata con approccio ibrido
def ner(text, labels: str, threshold: float, nested_ner: bool):
    labels = labels.split(",")
    sections = split_text_into_sections(text)  # Suddividi il testo in sezioni
    all_entities = []

    for section in sections:
        if len(section.split()) > 512:  # Se la sezione è lunga, usa sliding window
            segments = sliding_window(section)
        else:
            segments = [section]

        for segment in segments:
            entities = model.predict_entities(segment, labels, flat_ner=not nested_ner, threshold=threshold)
            formatted_entities = [
                {
                    "entity": entity["label"],
                    "word": entity["text"],
                    "start": entity["start"],
                    "end": entity["end"],
                    "score": entity["score"]  # Ora includiamo anche lo score originale
                }
                for entity in entities
            ]
            all_entities.extend(formatted_entities)

    # Unisci entità duplicate o frammentate
    merged_entities = merge_entities(all_entities)

    # Converti le entità nel formato atteso da HighlightedText
    # HighlightedText si aspetta [(text, label), ...] o format speciale
    gradio_format = []

    # Ordina le entità per posizione di inizio
    merged_entities.sort(key=lambda x: x["start"])

    # Trasforma nel formato atteso da Gradio
    for entity in merged_entities:
        gradio_format.append((entity["word"], entity["entity"]))

    return gradio_format  # Formato compatibile con Gradio HighlightedText
"""
# 5️⃣ Funzione che gestisce il caricamento del file e il NER
def process_file(file_obj, labels, threshold, nested_ner):
    if file_obj is None:
        return "Nessun file caricato", None

    text = extract_text_from_file(file_obj)
    if text.startswith("Errore"):
        return text, None

    result = ner(text, labels, threshold, nested_ner)
    return text, result
"""
# 5️⃣ Funzione che gestisce il caricamento del file e il NER
def process_file(file_obj, labels, threshold, nested_ner):
    if file_obj is None:
        return "Nessun file caricato", None

    text = extract_text_from_file(file_obj)
    if text.startswith("Errore"):
        return text, None

    result = ner(text, labels, threshold, nested_ner)
    return text, result
# UI con Gradio
examples = [
    [
        "Articolo 1: Oggetto del bando...",
        "person, location, date, organization, codice_CIG, data_scadenza",
        0.3,
        True,
    ],
]

with gr.Blocks(title="GLiNER-M-v2.1 con approccio ibrido") as demo:
    gr.Markdown(
        """
        # GLiNER con approccio ibrido
        Ora supporta il caricamento di bandi lunghi, con suddivisione intelligente, sliding window e post-processing.
        """
    )

    with gr.Tab("Caricamento file"):
        file_input = gr.File(label="Carica un file .txt", file_types=[".txt"])
        with gr.Row():
            labels = gr.Textbox(
                value="person, location, date, organization, codice_CIG, codice_CUP, data_pubblicazione, data_scadenza",
                label="Etichette",
                placeholder="Inserisci le etichette separate da virgola",
                scale=2,
            )
            threshold = gr.Slider(0, 1, value=0.3, step=0.01, label="Soglia", scale=1)
            nested_ner = gr.Checkbox(value=True, label="NER Annidate", scale=0)
        submit_btn = gr.Button("Analizza File")
        text_display = gr.Textbox(label="Testo Caricato", lines=10)
        output = gr.HighlightedText(label="Entità Rilevate")

    with gr.Tab("Input Manuale"):
        input_text = gr.Textbox(value=examples[0][0], label="Testo di input", lines=10)
        with gr.Row():
            manual_labels = gr.Textbox(value=examples[0][1], label="Etichette", scale=2)
            manual_threshold = gr.Slider(0, 1, value=0.3, step=0.01, label="Soglia", scale=1)
            manual_nested_ner = gr.Checkbox(value=examples[0][3], label="NER Annidate", scale=0)
        manual_submit_btn = gr.Button("Analizza Testo")
        manual_output = gr.HighlightedText(label="Entità Rilevate")

        gr.Examples(
            examples,
            fn=ner,
            inputs=[input_text, manual_labels, manual_threshold, manual_nested_ner],
            outputs=manual_output,
            cache_examples=True,
        )

    submit_btn.click(fn=process_file, inputs=[file_input, labels, threshold, nested_ner], outputs=[text_display, output])
    manual_submit_btn.click(fn=ner, inputs=[input_text, manual_labels, manual_threshold, manual_nested_ner], outputs=manual_output)

demo.queue()
demo.launch(debug=True)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Caching examples at: '/content/.gradio/cached_examples/768'
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c41f9ff35c4301d819.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c41f9ff35c4301d819.gradio.live


In [5]:
!pip install chardet
import re
import chardet
import nltk
from nltk.tokenize import word_tokenize

# Scarica il tokenizer di NLTK se necessario
nltk.download("punkt")

# Impostazioni per la segmentazione migliorata
MAX_LEN = 384  # Lunghezza massima di un segmento
OVERLAP = 100  # Sovrapposizione tra segmenti
PUNCTUATION = [".", "!", "?", ";", ":", ","]  # Punti di spezzatura preferiti
CHUNK_SIZE = 1024 * 1024  # Dimensione del blocco di lettura (1 MB)


# Funzione per tokenizzare il testo in parole senza dipendenze esterne
def simple_tokenize(text):
    return re.findall(r'\b\w+\b', text)


# Funzione per trovare il miglior punto di spezzatura
def find_best_split(words, max_len):
    """
    Cerca il miglior punto di spezzatura in base alla punteggiatura.
    """
    if len(words) <= max_len:
        return len(words)  # Nessuna spezzatura necessaria

    best_split = max_len  # Punto di spezzatura predefinito
    for i in range(max_len - 10, max_len - 50, -1):  # Cerca un punto vicino alla fine
        if words[i] in PUNCTUATION:
            best_split = i + 1  # Include la punteggiatura nel segmento
            break
    return best_split


# Percorsi file aggiornati
input_file_path = "/content/testo_input.txt"
output_file_path = "/content/testo_segmentato_ottimizzato.txt"


# Elaborazione del file in blocchi
with open(input_file_path, "r", encoding="ISO-8859-1") as f, open(output_file_path, "w", encoding="utf-8") as outfile:
    buffer = []  # Buffer per accumulare parole
    segment_count = 0

    while True:
        chunk = f.read(CHUNK_SIZE)
        if not chunk:
            break  # Fine del file

        # Tokenizza il blocco corrente
        words = simple_tokenize(chunk)
        buffer.extend(words)

        # Segmentazione del buffer
        while len(buffer) >= MAX_LEN:
            end = find_best_split(buffer, MAX_LEN)
            segment = buffer[:end]
            segment_count += 1
            outfile.write(
                f"Segment {segment_count} (Token Count: {len(segment)}):\n{' '.join(segment)}\n\n"
            )
            buffer = buffer[end - OVERLAP :]  # Sovrapposizione

    # Elabora eventuali parole rimanenti nel buffer
    if buffer:
        segment_count += 1
        outfile.write(
            f"Segment {segment_count} (Token Count: {len(buffer)}):\n{' '.join(buffer)}\n\n"
        )

print(f"Testo elaborato salvato in {output_file_path}")

Testo elaborato salvato in /content/testo_segmentato_ottimizzato.txt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
